In [97]:
import pandas as pd

inventory = pd.read_excel('files/inventory_rjo1.xlsx')

# Show all rows
#pd.set_option('display.max_rows', None)

# Show all columns
#pd.set_option('display.max_columns', None)

# Showing less columns/rows
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')

inventory

,MATERIAIS,QNTD. EM ESTOQUE,ESTOQUE MÍNIMO,STATUS
0,NaN,NaN,NaN,NaN
1,Fibra Óptica Duplex MM OM4 LC-UPC/LC-UPC,NaN,NaN,NaN
2,1.5 metros,11.0,10.0,ESTOQUE OK
3,3.0 metros,4.0,10.0,NECESSITA REPOSIÇÃO
4,5.0 metros,7.0,10.0,NECESSITA REPOSIÇÃO
...,...,...,...,...
81,Fibra Commscope 2.0M MM,1.0,1.0,ESTOQUE OK
82,Fibra Commscope 30.0M MM,1.0,1.0,ESTOQUE OK
83,Fibra IBM 5M MM,16.0,15.0,ESTOQUE OK
84,Bandeja Fixa 4 Pontos 1U,3.0,2.0,ESTOQUE OK


In [ ]:
### GENERAL ADJUSTS ###

# Dropping first row
inventory = inventory.iloc[1:]

# Renaming columns
inventory.rename(columns={'QNTD. EM ESTOQUE': 'qntd'}, inplace=True)
inventory.rename(columns={'ESTOQUE MÍNIMO': 'qntd_min'}, inplace=True)
inventory.rename(columns={'MATERIAIS': 'material'}, inplace=True)

# Lowercase all column names
inventory.columns = [col.lower() for col in inventory.columns]

# Converting Dtypes
inventory['qntd'] = inventory['qntd'].astype('Int64')
inventory['qntd_min'] = inventory['qntd_min'].astype('Int64')

# Dropping column "Status"
inventory.drop(columns=['status'], inplace=True)

# Removing 'metros' from column
inventory['material'] = inventory['material'].str.replace(' metros', '')

inventory

In [ ]:
### INVENTORY CABLES ###

# Separate just cables
cables_all = inventory.iloc[0:46]

# Create 'cable_type' column
cables_all['cable_type'] = cables_all['material'].where(
    cables_all['material'].str.contains('Fibra|Patch', case=False, na=False)
)

# Forward fill the cable type name to all rows below
cables_all['cable_type'] = cables_all['cable_type'].fillna(method='ffill')

# Remove rows that are only headers (where qntd and qntd_min are NA)
cables = cables_all.dropna(subset=['qntd', 'qntd_min'])

# Rename column 'material' to 'metragem'
cables.rename(columns={'material': 'metragem'}, inplace=True)

cables = cables[['cable_type', 'metragem', 'qntd', 'qntd_min']]

cables

In [ ]:
### REST OF MATERIALS ###

# Separate normal materials
materials_all = inventory.iloc[46:]

# Remove rows that are only headers (where qntd and qntd_min are NA)
materials = materials_all.dropna(subset=['qntd', 'qntd_min', 'material'])

# Create find rows that contain "Portas" in the 'material' column
mask_portas = materials["material"].str.contains("Portas", na=False)

# Add "Patch Panel - " prefix to the 'material' column for those rows
materials.loc[mask_portas, "material"] = "Patch Panel - " + materials.loc[mask_portas, "material"]

materials

In [ ]:
### TESTS ###
reposition = cables[cables['qntd']<cables['qntd_min']]
reposition



,cable_type,metragem,qntd,qntd_min
3,Fibra Óptica Duplex MM OM4 LC-UPC/LC-UPC,3.0,4,10
4,Fibra Óptica Duplex MM OM4 LC-UPC/LC-UPC,5.0,7,10
5,Fibra Óptica Duplex MM OM4 LC-UPC/LC-UPC,10.0,2,10
6,Fibra Óptica Duplex MM OM4 LC-UPC/LC-UPC,15.0,0,10
7,Fibra Óptica Duplex MM OM4 LC-UPC/LC-UPC,20.0,0,10
10,Fibra Óptica Duplex MM OM4 LC-UPC/LC-UPC,35.0,3,10
21,Fibra Óptica SM LC-UPC/LC-UPC,1.5,13,20
22,Fibra Óptica SM LC-UPC/LC-UPC,3,0,20
23,Fibra Óptica SM LC-UPC/LC-UPC,5,19,20
24,Fibra Óptica SM LC-UPC/LC-UPC,10,2,20
